In [1]:
import pathlib
import cv2
import numpy as np
import pandas as pd
import ast

import matplotlib.pyplot as plt
from skimage import img_as_float

def get_index_of_file(file_path, index):
    list_of_files = sorted(pathlib.Path(file_path).iterdir())
    f = list_of_files[index]
    try:
        f = int(f.stem)
    except ValueError:
        for current_file in list_of_files:
            if current_file.name[0] != '.':
                f = current_file
                break
        f = int(f.stem)
    return f

def get_files_list(file_path, transition_frames):
    files_list = list()
    start_int = get_index_of_file(file_path, 0)
    for i in transition_frames:
        files_list.append(str(pathlib.Path(file_path)
                      .joinpath('{:03d}.jpg'.format(i+start_int))))
    return files_list

def make_summary(image_list):
    images = []
    scaling = 0.2
    for image_file in image_list:
        img = cv2.imread(image_file, flags=cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, None,
                         fx=scaling, fy=scaling, 
                         interpolation=cv2.INTER_AREA)
        images.append(img)
        
    summary_image = np.concatenate(np.array(images), axis=1)
    return summary_image

def process_date(folder_date, t_frames_list):
    files_list = get_files_list(folder_date, t_frames_list)
    summary_img = make_summary(files_list)
    image_name = '/Volumes/NO NAME/sleeper/summary_images/{}.jpg'.format(folder_date.stem)
    cv2.imwrite(image_name,
               summary_img)

In [129]:
def process_all(frames_list_str='std_frames_list'):
    record = pd.read_csv('/Volumes/NO NAME/sleeper/record.csv', index_col=0)
    record['t_frames_list'] = record['t_frames_list'].apply(lambda x: ast.literal_eval(x))
    record['std_frames_list'] = record['std_frames_list'].apply(lambda x: ast.literal_eval(x))
    for index, row in record.iterrows():
        dirpath = '/Volumes/NO NAME/sleeper/photos_processed/'
        p = pathlib.Path(dirpath)
        folder = p.joinpath(row['folder_name'])
        t_frames_list = row[frames_list_str]
        process_date(folder, t_frames_list)

process_all('reduced_frames')

In [145]:
def reduce_continued_movements(image_list):
    reduced_list = []
    for i, f in enumerate(image_list):
        if i == 0:
            continue
        if (image_list[i] - image_list[i-1] > 1):
            reduced_list.append(image_list[i-1])
        else:
            continue
    if image_list[-1] - image_list[-2] > 1:
        reduced_list.append(image_list[-1])
    return reduced_list

ex = ast.literal_eval('[21, 92, 93, 94, 107, 132, 133, 143, 191, 194, 198]')
reduce_continued_movements(ex)

[21, 94, 107, 133, 143, 191, 194, 198]

In [162]:
record = pd.read_csv('/Volumes/NO NAME/sleeper/record.csv', index_col=0)
record['t_frames_list'] = record['t_frames_list'].apply(lambda x: ast.literal_eval(x))
record['std_frames_list'] = record['std_frames_list'].apply(lambda x: ast.literal_eval(x))

record['reduced_t_frames'] = (record['t_frames_list']
                               .apply(lambda x: reduce_continued_movements(x)))
record['reduced_std_frames'] = (record['std_frames_list']
                               .apply(lambda x: reduce_continued_movements(x)))
record['count_reduced_t_frames'] = (record['reduced_t_frames']
                               .apply(lambda x: len(x)))
record['count_reduced_std_frames'] = (record['reduced_std_frames']
                               .apply(lambda x: len(x)))

### Analysis for 

In [164]:
record[['count_reduced_t_frames', 'count_reduced_std_frames']]

,count_reduced_t_frames,count_reduced_std_frames
0,14,14
1,10,5
2,14,13
3,13,12
4,20,12
5,13,11
6,17,13
7,8,9
8,22,22
9,29,14


In [158]:
for index, row in record.iterrows():
    print('T__:{}|{}'.format(len(row['reduced_t_frames']), row['reduced_t_frames'] ))
    print('STD:{}|{}'.format(len(row['reduced_std_frames']), row['reduced_std_frames'] ))

T__:14|[2, 19, 26, 42, 50, 58, 72, 95, 110, 118, 165, 173, 202, 236]
STD:14|[1, 19, 26, 42, 50, 58, 72, 95, 110, 165, 173, 202, 236, 242]
T__:10|[0, 57, 60, 83, 120, 186, 208, 220, 222, 228]
STD:5|[120, 182, 220, 228, 245]
T__:14|[16, 68, 75, 94, 101, 119, 157, 176, 184, 196, 204, 208, 210, 214]
STD:13|[16, 68, 75, 94, 101, 119, 157, 176, 183, 202, 204, 208, 210]
T__:13|[0, 35, 53, 59, 112, 122, 124, 145, 186, 203, 207, 229, 232]
STD:12|[0, 35, 59, 112, 122, 145, 186, 203, 206, 229, 235, 237]
T__:20|[0, 50, 79, 98, 113, 116, 154, 156, 163, 175, 213, 256, 267, 272, 274, 277, 287, 290, 293, 295]
STD:12|[50, 79, 98, 154, 163, 175, 212, 266, 272, 287, 290, 293]
T__:13|[23, 67, 76, 90, 95, 118, 158, 177, 191, 196, 216, 251, 268]
STD:11|[21, 67, 76, 90, 158, 175, 177, 216, 251, 268, 270]
T__:17|[0, 12, 60, 93, 109, 139, 148, 165, 187, 261, 300, 306, 311, 314, 318, 325, 327]
STD:13|[0, 60, 93, 109, 139, 165, 261, 299, 306, 314, 318, 325, 327]
T__:8|[72, 98, 135, 162, 199, 214, 228, 230]
STD:9

In [160]:
for index, row in record.iterrows():
    print('STD:{}|{}'.format(len(row['std_frames_list']), row['std_frames_list'] ))
    print('RED:{}|{}'.format(len(row['reduced_std_frames']), row['reduced_std_frames'] ))

STD:21|[1, 19, 26, 42, 50, 58, 72, 95, 110, 162, 163, 164, 165, 173, 201, 202, 235, 236, 242, 244, 245]
RED:14|[1, 19, 26, 42, 50, 58, 72, 95, 110, 165, 173, 202, 236, 242]
STD:9|[119, 120, 182, 220, 225, 226, 227, 228, 245]
RED:5|[120, 182, 220, 228, 245]
STD:16|[16, 68, 74, 75, 94, 101, 119, 157, 176, 183, 201, 202, 204, 207, 208, 210]
RED:13|[16, 68, 75, 94, 101, 119, 157, 176, 183, 202, 204, 208, 210]
STD:14|[0, 35, 59, 112, 121, 122, 145, 186, 203, 206, 229, 234, 235, 237]
RED:12|[0, 35, 59, 112, 122, 145, 186, 203, 206, 229, 235, 237]
STD:20|[49, 50, 79, 98, 154, 163, 174, 175, 212, 266, 272, 285, 286, 287, 290, 292, 293, 298, 299, 300]
RED:12|[50, 79, 98, 154, 163, 175, 212, 266, 272, 287, 290, 293]
STD:18|[21, 67, 76, 89, 90, 158, 175, 177, 215, 216, 250, 251, 266, 267, 268, 270, 272, 273]
RED:11|[21, 67, 76, 90, 158, 175, 177, 216, 251, 268, 270]
STD:19|[0, 60, 93, 109, 139, 165, 260, 261, 299, 306, 313, 314, 317, 318, 322, 323, 324, 325, 327]
RED:13|[0, 60, 93, 109, 139, 165,

In [161]:
for index, row in record.iterrows():
    print('T__:{}|{}'.format(len(row['t_frames_list']), row['t_frames_list'] ))
    print('RT_:{}|{}'.format(len(row['reduced_t_frames']), row['reduced_t_frames'] ))

T__:25|[0, 1, 2, 19, 26, 42, 50, 58, 72, 95, 110, 118, 162, 163, 164, 165, 173, 201, 202, 235, 236, 242, 243, 244, 245]
RT_:14|[2, 19, 26, 42, 50, 58, 72, 95, 110, 118, 165, 173, 202, 236]
T__:23|[0, 56, 57, 60, 83, 119, 120, 182, 183, 184, 185, 186, 208, 219, 220, 222, 225, 226, 227, 228, 243, 244, 245]
RT_:10|[0, 57, 60, 83, 120, 186, 208, 220, 222, 228]
T__:21|[16, 68, 74, 75, 94, 101, 119, 157, 176, 183, 184, 196, 201, 202, 203, 204, 206, 207, 208, 210, 214]
RT_:14|[16, 68, 75, 94, 101, 119, 157, 176, 184, 196, 204, 208, 210, 214]
T__:20|[0, 35, 53, 59, 111, 112, 121, 122, 124, 145, 186, 203, 206, 207, 229, 232, 234, 235, 236, 237]
RT_:13|[0, 35, 53, 59, 112, 122, 124, 145, 186, 203, 207, 229, 232]
T__:31|[0, 49, 50, 79, 98, 113, 116, 154, 156, 163, 174, 175, 212, 213, 255, 256, 266, 267, 272, 274, 277, 285, 286, 287, 290, 292, 293, 295, 298, 299, 300]
RT_:20|[0, 50, 79, 98, 113, 116, 154, 156, 163, 175, 213, 256, 267, 272, 274, 277, 287, 290, 293, 295]
T__:29|[20, 21, 22, 23, 66, 